# Importar Librerias

In [14]:
import pandas as pd
import re
from numpy import mean
from numpy import median
from numpy import array
import matplotlib.pyplot as plt
from matplotlib import pyplot
import random
from math import sqrt
from sklearn.metrics import mean_squared_error

#
from warnings import catch_warnings
from warnings import filterwarnings

# La librería statsmodel tiene implementaciones de SARIMA y de Holt-Winters
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Si el computador tiene procesadores múltiples, se aprovecharán
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed



# Carga de Datos

In [3]:
df = pd.read_csv('./train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# 3) Limpieza y Procesamiento de datos

In [57]:
tweets = list(df.iloc[:,3])

#Lista de caracteres que se repiten
disallowed_characters = "._!:@#'=><?()|_-/[]*~;"

#Lista de stopwords, etiquetas HTML y combinaciones de \x89 que deben ser removidas
disallowed_lit = [' by ', ' there ', ' the ', ' and ', ' to ', '\n', '&amp', ' for ', ' nor ', 'yet', ' so ', ' but ', ' if ', 'if ', '&gt', '&lt', 'when', 'whenever', 'where', 'wherever', "while", ' for ', 'because', 'before', '\x89ûò', '\x89ûª', '\x89ûï', '\x89û÷', '\x89û', '\x89ã¢', '\x9d', '\x89ââ', '\x89ûó', '+']

for i in range(len(tweets)):
    
    #Cambiar todas las letras a minusculas
    tweets[i] = tweets[i].lower()
    
    #Eliminar links
    tweets[i] = re.sub('http://\S+|https://\S+', '', tweets[i])
    
    #Eliminar caracteres especiales
    for character in disallowed_characters:
        tweets[i] = tweets[i].replace(character, "")
    
    #Eliminar articulos, preposiciones y etiquetas HTML
    for character in disallowed_lit:
        tweets[i] = tweets[i].replace(character, " ")
    
    #Eliminar espacios que tienen tamaño superior a 1
    tweets[i] = re.sub("\s+" , " ", tweets[i])

#print(tweets)

df['cleaned_text'] = tweets

In [58]:
df.head()

,id,keyword,location,text,target,cleaned_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are reason of this earthquake may al...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked shelter in place are being...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive wildfires evacuation ord..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...
